In [5]:
import pandas as pd 
import json

# MTURK to COCO

In [ ]:
#final csv
CSV_PATH = "../LEGACY_test/JACKSON_APPROVED_10_5.csv"

#output path for JSON in COCO format, include file name
JSON_OUTPUT_PATH = "../LEGACY_test/JACKSON_APPROVED_10_5_coco.json"


df = pd.read_csv(CSV_PATH)


# only approved
df = df[df['jackson_approve'] == 'x']

json_out = {"info": {}, "licenses": [], "categories": [{"id":1,"name":"Officer"},{"id":2,"name":"Civilian"},{"id":3,"name":"Riot Shield"},{"id":4,"name":"Gun"},{"id":5,"name":"Pepper Spray"},{"id":6,"name":"Baton"}], "images": [], "annotations": []}
df.shape

In [ ]:
# update the images section
sub_df = df[['Input.image_url', 'Answer.annotatedResult.inputImageProperties.height', 'Answer.annotatedResult.inputImageProperties.width']]

i = 0
for row in sub_df.iterrows():
    json_out['images'].append({"id": i, "file_name": row[1][0].split('/')[-1], "height": row[1][1], "width": row[1][2]})
    i +=1

In [ ]:
#update the annotations section 

id = 0
image_id = 0

for item in df['Answer.annotatedResult.polygons'].iteritems():

    item = eval(item[1])

    #go through each annotation in one row
    for i in range(len(item)):

        #skip deleted annotations
        if item[i]['label'] == '':
            continue
            
        category_id = item[i]['label']
        category_id = [i['id'] for i in json_out['categories'] if category_id in i['name']][0]

        segmentation = []
    
        #go through each vertices in one annotation
        for j in item[i]['vertices']:
            
            segmentation.append(j['x'])
            segmentation.append(j['y'])

        #if list not empty    
        if bool(segmentation):
            segmentation.append(segmentation[0])
            segmentation.append(segmentation[1])

            #generate bboxes
            bbox = []
            highest_y = int
            highest_x = int
            lowest_y = int
            lowest_x = int
            for k in range(len(segmentation)):
                
                if k == 0:
                    topleft_pair = [segmentation[k], segmentation[k+1]]
                    highest_x = topleft_pair[0]
                    lowest_x = topleft_pair[0]
                    lowest_y = topleft_pair[1]
                    highest_y = topleft_pair[1]

                if k % 2:
                    continue
                
                if k + 1 <= len(segmentation):
                    pair = [segmentation[k], segmentation[k+1]]

                    if pair[0] > highest_x:
                        highest_x = pair[0]
                    elif pair[0] < lowest_x:
                        lowest_x = pair[0]

                    if pair[1] < lowest_y:
                        lowest_y = pair[1]
                    elif pair[1] > highest_y:
                        highest_y = pair[1]
      
        
            #POSSIBLE ERROR HERE. BY NESTING THIS LINE IN THE FOR LOOP HERE, IT MAY CAUSE ERROR BY NOT APPENDING AN ID WHEN EMPTY LIST 
            json_out['annotations'].append({"id": id, "iscrowd": 0, "image_id": image_id, "category_id":category_id, 
                    "segmentation": [segmentation], "bbox": [lowest_x, lowest_y, highest_x-lowest_x, highest_y-lowest_y], })

        id += 1
        
    image_id += 1




In [ ]:
# save json COCO to output path
with open(JSON_OUTPUT_PATH, 'w') as f:
    json.dump(json_out, f)

In [ ]:
json_out['annotations'][0]

In [ ]:
"""
COCO FORMAT:
{
    "info": {},
    "licenses": [],
    "categories": [
        ...
        {
            "id": 2,
            "name": "cat",
            "supercategory": "animal"
        },
        ...
    ],
    "images": [
        {
            "id": 0,
            "file_name": "<filename0>.<ext>",
            "height": 480,
            "width": 640,
        },
        ...
    ],
    "annotations": [
        {
            "id": 0,
            "image_id": 0,
            "category_id": 2,
            "bbox": [260, 177, 231, 199] ,
            "segmentation": [...],
            "area": 45969,
            "iscrowd": 0
        },
        ...
    ]
}

"categories": [{"id":1,"name":"Officer"}, ...]
"""

# COCO TO VIA VGG

In [ ]:
"""
Convert coco format into useable VIA/vgg format
"""


In [ ]:

json_file = open("../data/annotations/guns/0-873.json")
#json_file2 = open("")
j = json.load(json_file)
json_file.close()
#json_file2.close()

"""
COCO
{
"info": {},
"images": [{}, {}]
"annotations"[{}, {}]:
"categories":[{},{}]
}:

VGG
{ 'filename': '28503151_5b5b7ec140_b.jpg',
    'regions': {
        '0': {
            'region_attributes': {'name'=int},
            'shape_attributes': {
                'all_points_x': [...],
                'all_points_y': [...],
                'name': 'polygon'}},
        ... more regions ...
    },
    'size': 100202
    }
"""


In [ ]:

#Get ID's from coco's split data storage of annotations to match and into the same object

matching_ids = {}

for i, val in enumerate(j['annotations']):
     
    #add item label to url
    image_id = j['annotations'][i]['image_id']
    matching_ids[i] = []
    matching_ids[i].append(val)

    v = 0
    for u, val in enumerate(j['annotations']):
        if j['annotations'][u]['image_id'] == image_id and u != i:
            
            matching_ids[i].append(val)

#combine image_ids
image_id_list = []           
for i, val in enumerate(matching_ids.copy()):
    if j['annotations'][i]['image_id'] in image_id_list:
        del matching_ids[i]

    else:
        image_id_list.append(j['annotations'][i]['image_id'])
    
#make id match image_id
for i, val in enumerate(matching_ids.copy()):
    i = i+1

    matching_ids[i] = matching_ids[val]
    del matching_ids[val]
    
    
    

In [ ]:

#Assemble all annotations into a via/vgg list saved as "k"
k={}

for i, image in enumerate(j['images']):

    #filename
    file_name_id = {'file_name' : str, 'id': int, 'width': int, 'height': int, 'regions': {}}

    

    file_name_id['file_name'] = string +j['images'][i]['file_name'][0:63]
    file_name_id['id'] = j['images'][i]['id']
    file_name_id['width'] = j['images'][i]['width']
    file_name_id['height'] = j['images'][i]['height']
    #update with filename
    k.update({i: file_name_id})
    
    #add category and x and y
for ind, image in enumerate(k):
    
    for ind2, annot in enumerate(matching_ids[ind+1]):
        #go through each annotation for every image and append all x and y and category ids

        #get x and y points
        all_x =[]
        all_y = []

        for ind3, num in enumerate(annot['segmentation'][0]):
            if ind3 % 2 == 0:
                all_x.append(num)
            if ind3 % 2 == 1:
                all_y.append(num)
        # get first x and y to append at end of points to match via vgg format
        all_x.append(all_x[0])
        all_y.append(all_y[0])  

        #change name of category to string
        category = annot['category_id']
        if category == 1:
            category = 'Officer'
        if category == 2:
            category = 'Civilian'
        if category == 3:
            category = 'Gun'
        if category == 4:
            category = 'Baton'
        if category == 5:
            category = 'Riot Shield'
        if category == 6:
            category = 'Pepper Spray'

    #add first num to end
        k[ind]['regions'][ind2] = {'region_attributes': {'name': category}, 'shape_attributes': {'all_points_x': all_x,'all_points_y': all_y}}

     

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }


In [ ]:
#Split for training and val
val_len = round(len(k) * 0.177) #int with val size

val = dict(list(k.items())[len(k)-val_len:])
train = dict(list(k.items())[:len(k)-val_len])



In [ ]:
#save master, not train val split
with open('../data/annotations/masters/05_24_coco.json', 'w') as f:
    json.dump(k, f)
#Save K as JSON object
#update with path for upload
#train
"""with open('../source/Mask_RCNN/datasets/proto/train/via_region_data.json', 'w') as f:
    json.dump(train, f)
#val
with open('../source/Mask_RCNN/datasets/proto/val/via_region_data.json', 'w') as f:
    json.dump(val, f)"""

# Convert mturk csv to VIA VGG

In [ ]:
import pandas as pd
import json

In [ ]:
###########################################################################################################
# PARAMS
###########################################################################################################



#final csvs
final_csv = "../data/annotation_csvs/combined_csvs/MASTER_0707.csv"

#either g, ps, b for different objects, MAKE THIS STRING EMPTY FOR RIOT SHIELDS!!!!
starting_letter_id = ""


In [ ]:
df = pd.read_csv(final_csv)
obj = {} #dict to fill 

#loads a reference obj in perfect shape
json_file2 = open("../source/Mask_RCNN/datasets/proto/val/via_region_data.json")

ref = json.load(json_file2) #vgg annot
json_file2.close()

In [ ]:
len(df)

6334

In [ ]:
df = df.replace("nan", "")

#df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1','Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1'], axis=1)

df = df.fillna("")

approved = df#[df['Approve'] == 'x']

#randomize rows
df = df.sample(frac=1)

In [ ]:
#place format and file info into object

for i, row in approved.iterrows():
    image_name_loc = approved['Input.image_url'][i].find('images/') + 7
    file_name_id = {'file_name' : str, 'width': int, 'height': int, 'regions': {}}
    file_name_id['file_name'] = starting_letter_id + approved['Input.image_url'][i][image_name_loc:]
    file_name_id['height'] = int(approved['Answer.annotatedResult.inputImageProperties.height'][i])
    file_name_id['width'] = int(approved['Answer.annotatedResult.inputImageProperties.width'][i])

    #update with filename
    obj.update({i: file_name_id})   
    the_annot = eval(approved['Answer.annotatedResult.polygons'][i])


    """
    'regions': {'0': {'region_attributes': {'name': 'Civilian'},
        'shape_attributes': {'all_points_x': [141.53846153846155,
        141.53846153846155],
        'all_points_y': [224.6153846153846,
        224.6153846153846]}},
    '1': {'region_attributes': {'name': 'Civilian'},
        'shape_attributes': {'all_points_x': [777.4358974358975,
        777.4358974358975],
        'all_points_y': [205.47008547008548,
        205.47008547008548]}},
    '2': {'region_attributes': {'name': 'Gun'},
        'shape_attributes': {'all_points_x': [633.1623931623932,
        659.8290598290598],
        'all_points_y': [120.68376068376068]}}}}
    """
    


    for j in range(0, len(the_annot)):
        #contains a single classified annotation
        label = the_annot[j]['label']
        if label == '':
            continue
        all_x = []
        all_y = []
        
        #for every vertices inside a single label
        for v in the_annot[j]['vertices']:
            all_x.append(v['x'])
            all_y.append(v['y'])

        # get first x and y to append at end of points to match via vgg format

        #account for empty annotatins
        if all_x:
            all_x.append(all_x[0])
        else:
            pass
        if all_y:
            all_y.append(all_y[0])
        else:
            pass

        obj[i]['regions'][j] = {'region_attributes': {'name': label}, 'shape_attributes': {'all_points_x': all_x,'all_points_y': all_y}}



    #add first num to end
        #


In [ ]:
#Split for training and val
val_len = round(len(obj) * 0.177) #int with val size

val = dict(list(obj.items())[len(obj)-val_len:])
train = dict(list(obj.items())[:len(obj)-val_len])


In [ ]:
len(train)

5213

In [ ]:
#Save final object as JSON object
with open('../data/annotations/masters/0707_csv.json', 'w') as f:
    json.dump(obj, f)



#update with path for upload

#train
with open('../source/Mask_RCNN/datasets/mk2/train/via_region_data.json', 'w') as f:
    json.dump(train, f)
#val
with open('../source/Mask_RCNN/datasets/mk2/val/via_region_data.json', 'w') as f:
    json.dump(val, f)


# combine coco jsons

In [ ]:
import pandas as pd
import json

In [ ]:
# add names/paths of jsons you want to combine
jsons = ['general_0-1000-16-coco.json', 'general-0-129-1175-coco.json',
         'gun-0-873.json', 'JACKSON_APPROVED_10_5_coco.json', 'teargas_20000-22654-coco.json',
          'teargas_22654-23400-coco.json', 'teargas_23400-23453-coco.json']

# make a master json start as first element in jsons list
with open(jsons[0]) as json_file:
    master_json = json.load(json_file)


-



















-













In [ ]:

JSON_MASTER_PATH = "general_0-1000-16-coco.json"
JSON_PATH_2 = "general-0-129-1175-coco.json"

with open(JSON_MASTER_PATH) as json_file:
    json_1 = json.load(json_file)
with open(JSON_PATH_2) as json_file:
    json_2 = json.load(json_file)
 #vgg annot

max_len_1 = len(json_1) - 1

max_len_1

3

In [ ]:
#change keys of second json to start after 
new_json = {}

for key, val in json_2.items():
    new_json[str(max_len_1+1 + int(key))] = json_2[key]

json_2 = new_json

In [ ]:
merged_json = dict(json_1, **json_2)

In [ ]:
with open('../data/annotations/masters/0707_merged.json', 'w') as f:
    json.dump(merged_json, f)

# train val split a master VOC json


In [ ]:
import pandas as pd
import json

In [ ]:


JSON_PATH = "../data/annotations/masters/0707_merged.json"


with open(JSON_PATH) as json_file:
    master_json = json.load(json_file)


#Split for training and val
val_len = round(len(master_json) * 0.177) #int with val size
val = dict(list(master_json.items())[len(master_json)-val_len:])
train = dict(list(master_json.items())[:len(master_json)-val_len])

#update with path for upload
#train
with open('../source/Mask_RCNN/datasets/mk2/train/via_region_data.json', 'w') as f:
    json.dump(train, f)
#val
with open('../source/Mask_RCNN/datasets/mk2/val/via_region_data.json', 'w') as f:
    json.dump(val, f)

# Train Val Test Split COCO json


In [17]:
import json
from sklearn.model_selection import train_test_split

JSON_PATH = "completed_annotations/masters/master_mod.json"

with open(JSON_PATH) as json_file:
    master_json = json_file.readlines()

In [ ]:
train, val = train_test_split(master_json, test_size=0.3)